# Lectura 43: Funciones

## Condicionales

Polars admite condiciones similares a if-else en expresiones con la sintaxis `when`, `then`, y `otherwise`. La condición se coloca dentro de la cláusula `when` y cuando se evalúa como verdadero, se aplica lo que esté dentro del `then`; de lo contrario, se aplica lo que esté dentro del `otherwise`.

In [1]:
import polars as pl
from polars import col

vuelos_lf = (
    pl.scan_parquet('./data/vuelos/vuelos.parquet')
)

In [2]:
from polars import when, lit

vuelos_lf.select(
    col('MONTH'),
    when(col('MONTH') <= 6)
    .then(lit('Primer Semestre'))
    .otherwise(lit('Segundo Semestre'))
    .alias('semestre')
).collect()

MONTH,semestre
i32,str
1,"""Primer Semestre"""
1,"""Primer Semestre"""
1,"""Primer Semestre"""
1,"""Primer Semestre"""
1,"""Primer Semestre"""
…,…
12,"""Segundo Semestre"""
12,"""Segundo Semestre"""
12,"""Segundo Semestre"""


## `coalesce`

Esta función toma una o varias columnas como argumento y mantiene el primer valor no nulo de izquierda a derecha.

In [3]:
vuelos_lf.group_by('CANCELLATION_REASON').len().collect()

CANCELLATION_REASON,len
str,u32
"""B""",48851
"""A""",25262
null,5729195
"""C""",15749
"""D""",22


In [4]:
from polars import coalesce

vuelos_lf.select(
    col('*'),
    coalesce('CANCELLATION_REASON', lit('NA')).name.suffix('_DES')
).group_by('CANCELLATION_REASON_DES').len().collect()

CANCELLATION_REASON_DES,len
str,u32
"""NA""",5729195
"""D""",22
"""A""",25262
"""B""",48851
"""C""",15749


## `concat_list` y `concat_str`

Estas funciones permiten concatenar un conjunto de columnas en una lista o en un string.

In [5]:
from polars import concat_list, concat_str

vuelos_lf.select(
    concat_list('YEAR', 'MONTH', 'DAY')
).collect()

YEAR
list[i32]
"[2015, 1, 1]"
"[2015, 1, 1]"
"[2015, 1, 1]"
"[2015, 1, 1]"
"[2015, 1, 1]"
…
"[2015, 12, 31]"
"[2015, 12, 31]"
"[2015, 12, 31]"


In [6]:
vuelos_lf.select(
    concat_str('YEAR', 'MONTH', 'DAY', separator='-')
).collect()

YEAR
str
"""2015-1-1"""
"""2015-1-1"""
"""2015-1-1"""
"""2015-1-1"""
"""2015-1-1"""
…
"""2015-12-31"""
"""2015-12-31"""
"""2015-12-31"""
